<a href="https://colab.research.google.com/github/meretja/MasterThesis/blob/main/Master_Thesis_Code_Meret_Vivianne_Jaggi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Full archive search - Twitter

In [ ]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

# set a “TOKEN” variable
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAEkOlgEAAAAAeVHffsF0MpKg8khU6JnOoLpHk9M%3DdyGdF3vEqPCY4nBaiNlouPWXYBqCQAYjTT6PiFysIjiZ9fFYoo'

# retrieve the token from the environment.
def auth():
    return os.getenv('TOKEN')

# create headers to access the API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

# build request for endpoints
def create_url(keyword, start_date, end_date, max_results = 10):

    search_url = "https://api.twitter.com/2/tweets/search/all" # Change to the endpoint to collect data from

    #change params based on the used endpoint
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

# connect to endpoint
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

# Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "#mastercard lang:en"
start_time = "2023-01-27T00:00:00.000Z"
end_time = "2023-01-28T00:00:00.000Z"
max_results = 10

# Create URL and get response from the API
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

# print the response in a readable format
print(json.dumps(json_response, indent=4, sort_keys=True))

# retrieve the next_token
json_response['meta']['result_count']

# Save response as a csv-file

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count', 'tweet'])
csvFile.close()

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:

        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        #source = tweet['source']

        # 8. Tweet text
        text = tweet['text']

        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, text]

        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

append_to_csv(json_response, "data.csv")

## Loop over different time periods to group together certain number of tweets

In [ ]:
# specify start and end date
start_date = '2018-05-02'
end_date = '2020-08-13'

# generate a series of days
dates = pd.date_range(start_date, end_date)

start_dates = dates[0:-1]
start_dates = [d.strftime('%Y-%m-%dT%H:%M:%SZ') for d in start_dates]
# start_dates = start_dates.isoformat()
# start_dates = start_dates.strftime('%Y-%m-%dT%H:%M:%S.%f%z').tolist()

end_dates = dates[1:]
end_dates = [d.strftime('%Y-%m-%dT%H:%M:%SZ') for d in end_dates]
# end_dates = end_dates.strftime('%Y-%m-%dT%H:%M:%S.%f%z').tolist()


#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "$abbv lang:en"

max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("abbv1.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_dates)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None

    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_dates[i],end_dates[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_dates[i])
                append_to_csv(json_response, "abbv1.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(2)
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_dates[i])
                append_to_csv(json_response, "abbv1.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(2)

            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(2)
print("Total number of results: ", total_tweets)

# Sentiment extraction from sourced tweets

Exemplary code of sentiment extraction using TSLA ticker
The same was done for each individual ticker, individually, to assign sentiment to each Tweet that was scraped previously

In [ ]:
!pip install transformers

"""Load packages and pre-trained model from Hugging Face"""

import os
import pandas as pd

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

"""Define FinBERT model and sentiment analysis task"""

"""
Defining FinBERT Model (yiyanghkust/finbert-tone) to yield the following results with the corresponding confidence level:
#LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative
"""

 # use pre-trained yiyanghkust/finbert-tone model and tokeniser
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

# define task to be sentiment analysis and set model and tokeniser, accordingly
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

"""Load tweets"""

# Set path to csv-files of tweets
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Preparing tweets to feed them to the pre-trained model
tsla = pd.read_csv('/content/drive/MyDrive/tweets/TSLA.csv')
tsla['tweet'] = 'tsla'
tsla = tsla.rename(columns={"tweet": "ticker", "source": "text"})
tsla = tsla.drop(columns=["author id", "geo", "id", "lang", "like_count", "quote_count", "reply_count", "retweet_count"])

"""add extra code to separate the dataframe into two parts --> can only run for 24h and these datasets are too large for that"""

# tsla1 = tsla.iloc[:260000,:]
# tsla2 = tsla.iloc[260000:520000,:]
tsla3 = tsla.iloc[520000:,:]

"""Apply NLP function to the tweets"""

results_tsla3 = []
for tweet in tsla3["text"]:
    result = nlp(tweet)
    results_tsla3.append(result)

"""Add sentiment and confidence level to DataFrame"""

# Extract the sentiment scores and confidence levels from each dictionary
sentiment_tsla3 = [result[0]["label"] for result in results_tsla3]
confidence_tsla3 = [result[0]["score"] for result in results_tsla3]

# Add the sentiment scores and confidence levels to the DataFrame
tsla3["sentiment_score"] = sentiment_tsla3
tsla3["confidence_level"] = confidence_tsla3

"""Convert to csv-file"""

tsla3.to_csv('/content/drive/MyDrive/sentiment_tsla3.csv', index=False)

tsla3.head()

# MAIN ANALYSIS

---



The following file includes the computation of sentiment measures, z-scores, changes in demand as well as conducting the entire set of analyses from the thesis

# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from functools import reduce
import matplotlib.dates as mdates
from pykalman import KalmanFilter
from scipy.optimize import minimize
from tabulate import tabulate
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
from pystout import pystout
from scipy import stats
from linearmodels import PanelOLS
from linearmodels.panel import compare
import seaborn as sns
import warnings
import hmmlearn
from scipy.stats import ttest_ind

# define colours for the subsequent illustrations
colors = sns.color_palette('Set2', n_colors=3)

# Sentiment Scores

## Import sentiment data scraped from Twitter and stored in .csv files

In [ ]:
path = '/Users/meret/Desktop/Uni/Master_Thesis/Data/sentiment_extraction'
filenames = glob.glob(path + "/*.csv")

dfs = []

for filename in filenames:
    temp_df = pd.read_csv(filename)
    dfs.append(temp_df)

# transform list of dataframes into one single dataframe
sentiment_data = pd.concat(dfs)

# create boolean mask to delete any entries in the "created_at" column that are not a date (i.e. not 25 characters long)
sentiment_data['created_at'] = sentiment_data['created_at'].fillna(-1)
mask = sentiment_data['created_at'].apply(lambda x: len(str(x))) == 25
sentiment_data = sentiment_data.loc[mask]

sentiment_data['created_at'] = pd.to_datetime(sentiment_data['created_at'])
sentiment_data['date'] = sentiment_data['created_at'].dt.date
sentiment_data = sentiment_data.drop(columns=['created_at'])

## Sort confidence_level and inspect number of entries below a certain threshold value

In [ ]:
# Check whether there are string values in confidence_level
string_values = sentiment_data['confidence_level'].apply(lambda x: isinstance(x, str))

# RESULT: over a million entries have a string instead of float value
    # --> convert to float value in order to sort them

# Convert string values to float
sentiment_data['confidence_level'] = pd.to_numeric(sentiment_data['confidence_level'], errors='coerce')

# Count entries that have confidence level below certain threshold values
below_threshold_fifty = (sentiment_data['confidence_level'] < 0.5).sum()    # sum: 4'871
below_threshold_seventy = (sentiment_data['confidence_level'] < 0.7).sum()  # sum: 142'281
below_threshold_ninety = (sentiment_data['confidence_level'] < 0.9).sum()   # sum: 356'948

## Frequency of sentiment categories per day

In [ ]:
negative = sentiment_data[sentiment_data['sentiment_score'] == 'Negative'].groupby(['date', 'ticker'])['sentiment_score'].value_counts().sort_index()
negative = negative.rename('negative').reset_index()[['date', 'ticker', 'negative']]
negative = negative.set_index('date')

neutral = sentiment_data[sentiment_data['sentiment_score'] == 'Neutral'].groupby(['date', 'ticker'])['sentiment_score'].value_counts().sort_index()
neutral = neutral.rename('neutral').reset_index()[['date', 'ticker', 'neutral']]
neutral = neutral.set_index('date')

positive = sentiment_data[sentiment_data['sentiment_score'] == 'Positive'].groupby(['date', 'ticker'])['sentiment_score'].value_counts().sort_index()
positive = positive.rename('positive').reset_index()[['date', 'ticker', 'positive']]
positive = positive.set_index('date')

# Combine dataframes
sentiment_list = [neutral, negative, positive]
sentiment = reduce(lambda left,right: pd.merge(left,right,on=['date','ticker'], how='outer'), sentiment_list)
sentiment.fillna(0, inplace=True)

## Add sentiment-weighted count-based measure

In [ ]:
sentiment = sentiment.reset_index()

for index, row in sentiment.iterrows():

    neut = row['neutral']
    neg = row['negative']
    pos = row['positive']

    # Calculate weighted count-based measure
    weighted = (neg*0.5) + (neut*1) + (pos*1.5)
    weighted_avg = weighted / (neg + neut + pos)

    sentiment.at[index, 'weighted'] = weighted
    sentiment.at[index, 'weighted_avg'] = weighted_avg

## Deal with missing BRK.B entries

In [ ]:
sentiment = sentiment.sort_values(by=['ticker', 'date'])
grouped_data = sentiment.groupby('ticker')
grouped_data.size() # shows that BRK.B has a few entries missing

brkb_sentiment = sentiment[sentiment['ticker'] == 'brkb'].iloc[:,:]

# create DataFrame of 834 dates
sentiment = sentiment.set_index('date')
date_range = pd.date_range(start=sentiment.index.min(), end=sentiment.index.max(), freq='D')
date_df = pd.DataFrame({'date': date_range})

# transform both date series to datetime64 to merge
date_df['date'] = pd.to_datetime(date_df['date'])
brkb_sentiment['date'] = pd.to_datetime(brkb_sentiment['date'])

# add entry for each day in the observation period
merged_brkb = pd.merge(brkb_sentiment, date_df, on='date', how='outer')
merged_brkb['ticker'].fillna('brkb', inplace=True)
merged_brkb.fillna(0, inplace=True)
merged_brkb['weighted_avg'] = merged_brkb['weighted_avg'].replace(0, method='ffill')

# add new BRKB series to original sentiment dataframe
sentiment = sentiment.reset_index()
sentiment = sentiment.drop(sentiment[sentiment['ticker'] == 'brkb'].index)
sentiment['date'] = pd.to_datetime(sentiment['date'])
sentiment = pd.concat([sentiment, merged_brkb])

sentiment = sentiment.sort_values(by=['ticker', 'date'])
sentiment = sentiment.set_index('date')

# check that we now have enough entries for each ticker
grouped_data = sentiment.groupby('ticker')
grouped_data.size()

# Kalman filter

## Define Kalman filter function

In [ ]:
def kalman_log_likelihood(params, sentiment_scores):
    # Unpack the parameters
    initial_state_mean, initial_state_covariance, observation_covariance, transition_matrices, transition_covariance = params

    # Create the Kalman filter
    kf = KalmanFilter(initial_state_mean=initial_state_mean,
                      initial_state_covariance=initial_state_covariance,
                      observation_covariance=observation_covariance,
                      transition_matrices=transition_matrices,
                      transition_covariance=transition_covariance)

    # Calculate the log-likelihood of the observed sentiment scores
    _, log_likelihoods = kf.filter(sentiment_scores)

    return -np.sum(log_likelihoods)

## Create filtered scores

In [ ]:
for ticker, stock_data in grouped_data:

    # Extract the sentiment scores for the current stock
    sentiment_scores = sentiment['weighted_avg'].values

    # Estimate the Kalman filter parameters for the current stock using
        # Maximum Likelihood Estimation (minimise negative log-likelihood)

    initial_guess = (0.0, np.eye(1), np.eye(1), np.eye(1), np.eye(1))  # Initial guess for the parameters
    bounds = ((None, None), (None, None), (None, None), (None, None), (None, None))  # Bounds for the parameters
    result = minimize(kalman_log_likelihood, initial_guess, args=(sentiment_scores,), bounds=bounds)
    estimated_params = result.x

    # Unpack the estimated parameters
    estimated_initial_state_mean, estimated_initial_state_covariance, estimated_observation_covariance, \
    estimated_transition_matrices, estimated_transition_covariance = estimated_params

    # Apply the estimated parameters to the Kalman filter
    kf = KalmanFilter(initial_state_mean=estimated_initial_state_mean,
                      initial_state_covariance=estimated_initial_state_covariance,
                      observation_covariance=estimated_observation_covariance,
                      transition_matrices=estimated_transition_matrices,
                      transition_covariance=estimated_transition_covariance)

    # Apply the Kalman filter to the sentiment scores for the current stock
    filtered_scores, _ = kf.filter(sentiment_scores)

    sentiment['filtered_scores'] = filtered_scores

Individual scores (Appendix)

In [ ]:
sentiment = sentiment.reset_index()
unique_tickers = sentiment['ticker'].unique()
ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):
    fig, axes = plt.subplots(5, 2, figsize=(12, 15))

    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    # Loop through each ticker in the current group
    for idx, ticker in enumerate(ticker_group):
        # Get the data for the current ticker
        ticker_data = sentiment[sentiment['ticker'] == ticker]

        # Plot the pct_change for the current ticker
        ax = axes[idx]
        ax.plot(ticker_data['date'], ticker_data['weighted_avg'], 'red', label='Unfiltered Scores', linewidth=0.6)
        ax.plot(ticker_data['date'], ticker_data['filtered_scores'], 'aqua', label='Filtered Scores', linewidth=0.6)
        ax.set_title(ticker)
        ax.set_ylabel('Score')
        ax.grid(True)

        # Format x-axis tick labels to display only years
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.setp(ax.xaxis.get_majorticklabels())

    for ax in axes:
        ax.set_title(ax.get_title(), fontsize=10, fontweight='bold')

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    plt.savefig(f"scores_{group_idx+1}.png")
    plt.show()

TSLA example

In [ ]:
mean_sentiment_score = sentiment.groupby('date')['filtered_scores'].mean().reset_index()

plt.style.use('seaborn')
plt.figure(figsize=(12, 6))
plt.plot(tsla_sentiment.date, tsla_sentiment.weighted_avg, 'red', label='Unfiltered Scores', linewidth=0.6)
plt.plot(tsla_sentiment.date, tsla_sentiment.filtered_scores, 'cyan', label='Filtered Scores', linewidth=0.6)
plt.title('Raw vs. Filtered Sentiment Scores - TSLA')
plt.legend()
plt.show()

## Test statistical difference between raw vs. filtered score

In [ ]:
for ticker in grouped_data:
    unfiltered_scores = sentiment['weighted_avg']
    filtered_scores = sentiment['filtered_scores']

    # Normality test (Shapiro-Wilk test)
    _, unfiltered_pvalue = stats.shapiro(unfiltered_scores)
    _, filtered_pvalue = stats.shapiro(filtered_scores)

    # Perform the appropriate test based on normality results
    if unfiltered_pvalue > 0.05 and filtered_pvalue > 0.05:

        # Perform paired t-test
        _, pvalue = stats.ttest_rel(unfiltered_scores, filtered_scores)
        test_name = "Paired t-test"

    else:
        # Perform Wilcoxon signed-rank test
        _, pvalue = stats.wilcoxon(unfiltered_scores, filtered_scores)
        test_name = "Wilcoxon signed-rank test"

    print("Stock:", ticker[0])
    print("Normality test p-values:")
    print(test_name + " p-value:", pvalue)
    print()

## Distribution of sentiment scores

Individual tickers

In [ ]:
unique_tickers = sentiment['ticker'].unique()
ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):

    fig, axes = plt.subplots(5, 2, figsize=(15, 20))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        ticker_data = sentiment[sentiment['ticker'] == ticker]
        filtered_scores = ticker_data['filtered_scores']

        # Plot the histogram for the current ticker
        ax = axes[idx]
        ax.hist(filtered_scores, bins=100, density=True, color='darkgreen')
        ax.set_title(f"Distribution of Sentiment Scores -- {ticker}")
        ax.set_xlabel("Filtered Scores")
        ax.set_ylabel("Density")

    for i in range(len(ticker_group), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"distribution_sentiment_ind_{group_idx+1}.png")
    plt.show()

Average score

In [ ]:
plt.hist(mean_sentiment_score['filtered_scores'], bins=100, color=colors[0], linewidth=0.1)
plt.title('Distribution of Average Sentiment Scores')
plt.savefig('distribution_sentiment_avg.png')
plt.show()

## Visualise time series of scores

In [ ]:
# TSLA positive vs. negative and weighted score

tsla_sentiment = sentiment[sentiment['ticker'] == 'tsla'].iloc[1:,:]

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(tsla_sentiment.negative, 'r', label='Negative', linewidth=0.3)
#plt.plot(tsla_sentiment.neutral, 'b', label='Neutral', linewidth=0.3)
plt.plot(tsla_sentiment.positive, 'g', label='Positive', linewidth=0.3)
plt.title('Sentiment TSLA')
plt.legend()
plt.show()

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(tsla_sentiment.date, tsla_sentiment.filtered_scores, linewidth=0.3)
plt.title('Weighted Average of Sentiment - TSLA')
plt.legend()
plt.show()

# LLY positive vs. negative and weighted score

lly_sentiment = sentiment[sentiment['ticker'] == 'lly'].iloc[1:,:]

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(lly_sentiment.negative, 'r', label='Negative', linewidth=0.3)
#plt.plot(lly_sentiment.neutral, 'b', label='Neutral', linewidth=0.3)
plt.plot(lly_sentiment.positive, 'g', label='Positive', linewidth=0.3)
plt.title('Sentiment LLY')
plt.legend()
plt.show()

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(lly_sentiment.date, lly_sentiment.filtered_scores, linewidth=0.3)
plt.title('Weighted Average of Sentiment - LLY')
plt.legend()
plt.show()

# Aggregate Data

mean_sentiment_data = sentiment.groupby('date')[['neutral', 'positive', 'negative', 'filtered_scores']].mean().reset_index()

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(mean_sentiment_data.date, mean_sentiment_data.negative, 'r', label='Negative', linewidth=0.5)
#plt.plot(lly_sentiment.neutral, 'b', label='Neutral', linewidth=0.3)
plt.plot(mean_sentiment_data.date, mean_sentiment_data.positive, 'g', label='Positive', linewidth=0.5)
plt.title('Aggregate Sentiment')
plt.legend()
plt.show()

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(mean_sentiment_data.date, mean_sentiment_data.filtered_scores, linewidth=0.3)
plt.title('Weighted Average of Aggregate Sentiment')
plt.legend()
plt.show()

## Summary of sentiment scores

In [ ]:
# for entire data set
summary_table = sentiment.describe()
sentiment[['neutral', 'negative', 'positive']].describe()
sentiment['filtered_scores'].describe()

# for average sentiment scores
mean_sentiment_score['filtered_scores'].describe()

# for individual tickers
summary_stats_list = []

for ticker, ticker_data in grouped_data:
    filtered_scores = ticker_data['filtered_scores']

    summary_stats = filtered_scores.describe().to_frame().T
    summary_stats['Ticker'] = ticker
    summary_stats_list.append(summary_stats)

summary_table_ind = pd.concat(summary_stats_list, ignore_index=True)
summary_table_ind = summary_table_ind[['Ticker'] + summary_table.columns[:-1].tolist()]
summary_table_ind = summary_table_ind.round(2)

# Convert the summary table to LaTeX format
latex_table = tabulate(summary_table_ind, tablefmt='latex', headers='keys', showindex=False)
print(latex_table)

# Z-score

## Define z-score function

In [ ]:
def rollingzscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

## Aggregate scores

In [ ]:
weekly_zScore = rollingzscore(mean_sentiment_score['filtered_scores'], 7)
weekly_zScore = weekly_zScore.rename('weekly')
monthly_zScore = rollingzscore(mean_sentiment_score['filtered_scores'], 30)
monthly_zScore = monthly_zScore.rename('monthly')

aggregate_z_scores = pd.concat([weekly_zScore, monthly_zScore], axis=1)
aggregate_z_scores['date'] = mean_sentiment_score['date']
aggregate_z_scores = aggregate_z_scores.set_index('date').reset_index()
aggregate_z_scores = aggregate_z_scores.dropna()

# Describe
description_overall_zScores = aggregate_z_scores.describe()

# Visualise

# Distribution
plt.figure(figsize=(10,6))
plt.hist([aggregate_z_scores['weekly'], aggregate_z_scores['monthly']], label=['Weekly', 'Monthly'], bins=60, color=[colors[0], colors[1]])
plt.title('Distribution of Aggregate Z-Scores')
plt.legend(fontsize=10, frameon=True)
plt.savefig("daily_average_zScores_hist.png")
plt.show()

# Time series
plt.figure(figsize=(10,6))
plt.plot(aggregate_z_scores.date, aggregate_z_scores['weekly'], label='Weekly', color=colors[0], linewidth=0.7)
plt.plot(aggregate_z_scores.date, aggregate_z_scores['monthly'], label='Monthly', color=colors[1], linewidth=0.7)
plt.title('Time Series of Aggregate Z-Scores')
plt.legend(fontsize=10, frameon=True)
plt.savefig("daily_average_zScores_plot.png")
plt.show()

## Individual tickers

In [ ]:
# compute z-scores for each ticker, individually
z_scores_ind = pd.DataFrame()
sentiment  = sentiment.reset_index()

for ticker in sentiment['ticker'].unique():

    ticker_data = sentiment[sentiment['ticker'] == ticker].copy()

    ticker_data['z_scores_weekly'] = rollingzscore(ticker_data['filtered_scores'], 7)
    ticker_data['z_scores_monthly'] = rollingzscore(ticker_data['filtered_scores'], 30)

    z_scores_ind = z_scores_ind.append(ticker_data[['date', 'ticker', 'z_scores_weekly', 'z_scores_monthly']], ignore_index=True)

z_scores_ind = z_scores_ind.sort_values(by=['ticker', 'date'])

# Summarise
z_scores_ind_weekly = z_scores_ind[['date', 'ticker', 'z_scores_weekly']].dropna()
z_scores_ind_monthly = z_scores_ind[['date', 'ticker', 'z_scores_monthly']].dropna()

description_ind_zScores_weekly = z_scores_ind_weekly.groupby('ticker').describe()
description_ind_zScores_monthly = z_scores_ind_monthly.groupby('ticker').describe()


# Visualise using plot
unique_tickers = z_scores_ind['ticker'].unique()
ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):
    fig, axes = plt.subplots(5, 2, figsize=(12, 15))

    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        ticker_data = z_scores_ind[z_scores_ind['ticker'] == ticker]

        dates = ticker_data['date']
        weekly = ticker_data[['date', 'ticker', 'z_scores_weekly']].set_index('date')
        monthly = ticker_data[['date', 'ticker', 'z_scores_monthly']].set_index('date')

        ax = axes[idx]
        ax.plot(dates, weekly['z_scores_weekly'], color=colors[0], label='Weekly Z-Score', linewidth=0.5, alpha=0.8)
        ax.plot(dates, monthly['z_scores_monthly'], color=colors[1], label='Monthly Z-Score', linewidth=0.5, alpha=0.7)
        ax.set_title(ticker)
        ax.set_ylabel('Z-Scores', fontsize=8)
        ax.xaxis.set_tick_params(labelsize=8)
        ax.legend(fontsize=8, frameon=True)

        ax.grid(True)

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    plt.savefig(f"z_scores_{group_idx+1}.png")
    plt.show()

# Robinhood Holdings

## Import data sources from Robintrack

In [ ]:
path = '/Users/meret/Desktop/Uni/Master_Thesis/Data/robinhood/selected_stocks'

filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    temp_df = pd.read_csv(filename)
    temp_df['ticker'] = filename[70:-4]
    dfs.append(temp_df)

# Put together all Robinhood holdings
robinhood_holdings = pd.concat(dfs, ignore_index=True)
robinhood_holdings['date'] = pd.to_datetime(robinhood_holdings['timestamp']).dt.date
robinhood_holdings = robinhood_holdings.drop(columns=['timestamp'])

# compute daily mean of holdings
robinhood_holdings = robinhood_holdings.groupby(['ticker', 'date']).mean()

# set 'date' as index
robinhood_holdings = robinhood_holdings.reset_index()

# make ticker lowercase to match other dataframes
robinhood_holdings['ticker'] = robinhood_holdings['ticker'].str.lower()

# rename brk.b to brkb in robinhood data to match sentiment data
robinhood_holdings['ticker'] = robinhood_holdings['ticker'].replace('brk.b', 'brkb')

## Deal with missing BRK.B entries

In [ ]:
# fill missing entries of brkb with zero
brkb_holdings = robinhood_holdings[robinhood_holdings['ticker'] == 'brkb'].iloc[:,:]

# create DataFrame with only the entire 834 dates
date_range = pd.date_range(start=robinhood_holdings['date'].min(), end=robinhood_holdings['date'].max(), freq='D')
date_df = pd.DataFrame({'date': date_range})

# transform both date series to datetime64 to merge
date_df['date'] = pd.to_datetime(date_df['date'])
brkb_holdings['date'] = pd.to_datetime(brkb_holdings['date'])

# add entry for each day in the observation period
merged_brkb = pd.merge(brkb_holdings, date_df, on='date', how='outer')
merged_brkb['ticker'].fillna('brkb', inplace=True)
merged_brkb.fillna(0, inplace=True)

# add new BRKB series to original sentiment dataframe
robinhood_holdings = robinhood_holdings.drop(robinhood_holdings[robinhood_holdings['ticker'] == 'brkb'].index)
robinhood_holdings = pd.concat([robinhood_holdings, merged_brkb])

robinhood_holdings['date'] = pd.to_datetime(robinhood_holdings['date'])
robinhood_holdings = robinhood_holdings.sort_values(by=['ticker', 'date'])
robinhood_holdings = robinhood_holdings.set_index('date')

## Create change in holdings

In [ ]:
# compute change in holdings and save in new dataframe
robinhood_holdings['pct_change'] = robinhood_holdings.groupby('ticker')['users_holding'].pct_change()
robinhood_holdings = robinhood_holdings.reset_index()
holdings_change = robinhood_holdings[['date', 'ticker', 'pct_change']]

# count and delete NaN from pct_change computation and replace with 0
holdings_change.isna().sum()
holdings_change = holdings_change[holdings_change.date != '2018-05-02 00:00:00'] # delete first observation with NaN from taking pct_change
holdings_change.isna()
holdings_change = holdings_change.fillna(0)

# delete observations where 'pct_change' == inf by first sorting values and dropping last row
holdings_change = holdings_change.sort_values(by='pct_change')[:-1]
holdings_change = holdings_change.sort_values(by=['ticker', 'date'])

## Plot changes in holdings

Individual tickers

In [ ]:
unique_tickers = holdings_change['ticker'].unique()
ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):

    fig, axes = plt.subplots(5, 2, figsize=(12, 15))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        ticker_data = holdings_change[holdings_change['ticker'] == ticker]

        ax = axes[idx]
        ax.plot(ticker_data['date'], ticker_data['pct_change'], linewidth=0.5)
        ax.set_title(ticker)
        ax.set_ylabel('Pct Change')
        ax.grid(True)

        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.setp(ax.xaxis.get_majorticklabels())

    for ax in axes:
        ax.set_title(ax.get_title(), fontsize=10, fontweight='bold')

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    plt.savefig(f"group_{group_idx+1}_tickers.png")
    plt.show()


### Examples - TSLA and LLY

In [ ]:
# TSLA
tsla_demand = holdings_change[holdings_change['ticker'] == 'tsla'].iloc[1:,:]
plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(tsla_demand.date, tsla_demand['pct_change'], color=colors[0], linewidth=0.6)
plt.title('Change in Holdings of TSLA Stock')
plt.legend()
plt.show()

# LLY
lly_demand = holdings_change[holdings_change['ticker'] == 'lly'].iloc[1:,:]
plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(lly_demand.date, lly_demand['pct_change'], color=colors[0],linewidth=0.6)
plt.title('Change in Holdings of LLY Stock')
plt.legend()
plt.show()

# Correlation Analysis

### Create correlation data

In [ ]:
correlation_data = holdings_change.merge(sentiment[['date', 'ticker', 'filtered_scores']], on=['date', 'ticker'])
# check for NaN --> should be none
correlation_data.isna().sum()
# group by ticker
grouped_corr_data = correlation_data.groupby('ticker')

# create new data frame where filtered scores are shifted by one
    # aimed to test how 1-day lagged sentiment is correlated with change in demand

shifted_data = pd.DataFrame()

# Loop through each ticker in the grouped data for one-day lag
for ticker, group_df in grouped_corr_data:

    group_df['shifted_scores'] = group_df['filtered_scores'].shift(periods=1)
    shifted_data = pd.concat([shifted_data, group_df], ignore_index=True)

# drop NaNs generated by the shift
shifted_data.isna().sum()
shifted_data = shifted_data.dropna()

### Lagged correlation

In [ ]:
# create empty list to store correlation results
lagged_correlation_results = []
grouped_corr_data = shifted_data.groupby('ticker')

window_size = 7

for ticker, group in grouped_corr_data:

    # normalise data
    mean_change = np.mean(group['pct_change'])
    norm_change = group['pct_change'] - mean_change
    mean_score = np.mean(group['shifted_scores'])
    norm_score = group['shifted_scores'] - mean_score

    # compute cross correlation between change in holdings and sentiment score
    rolling_corr = norm_change.rolling(window=window_size).corr(norm_score)

    cross_corr = np.correlate(group['pct_change'], group['shifted_scores'], mode='same')

    # norm_cross_corr = np.correlate(norm_change, norm_score)[0]
    # norm_cross_corr /= (len(norm_change) * group['pct_change'].std() * group['shifted_scores'].std())

    temp_df = pd.DataFrame()
    temp_df['ticker'] = group['ticker']
    temp_df['date'] = group['date']
    temp_df['corr_coeff'] = cross_corr
    temp_df['rolling_corr'] = rolling_corr

    lagged_correlation_results.append(temp_df)

lagged_correlation_results = pd.concat(lagged_correlation_results, ignore_index=True)
lagged_correlation_results = lagged_correlation_results[['date', 'ticker', 'rolling_corr']]

# drop NaNs and replace infinity with zeros
lagged_correlation_results = lagged_correlation_results.dropna()
lagged_correlation_results.replace([np.inf, -np.inf], 0, inplace=True)

### Plot lagged correlation coefficients over time

Individual stocks

In [ ]:
unique_tickers = lagged_correlation_results['ticker'].unique()
ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):

    fig, axes = plt.subplots(5, 2, figsize=(12, 15))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        ticker_data = lagged_correlation_results[lagged_correlation_results['ticker'] == ticker]

        # Plot the correlation coefficient for the current ticker
        ax = axes[idx]
        ax.plot(ticker_data['rolling_corr'], linewidth=0.5, color=colors[0])

        # Add polynomial trend line of degree 3
        coeffs = np.polyfit(ticker_data.index, ticker_data['rolling_corr'], 3)
        trend_line = np.polyval(coeffs, ticker_data.index)
        ax.plot(ticker_data.index, trend_line, color=colors[1], linewidth=0.8)

        ax.set_title(ticker)
        ax.set_ylabel('Lagged Correlation Coefficient')
        ax.set_xticklabels([])

        ax.grid(True)

    for ax in axes:
        ax.set_title(ax.get_title(), fontsize=10, fontweight='bold')

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    plt.savefig(f"rolling_corr{group_idx+1}.png")
    plt.show()


Example of TSLA

In [ ]:
lag_corr_tsla = lagged_correlation_results[lagged_correlation_results['ticker'] == 'tsla']
max_corr_row_tsla = lag_corr_tsla.loc[lag_corr_tsla['rolling_corr'].idxmax()]
max_corr_index_tsla = max_corr_row_tsla['date']

plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(lag_corr_tsla['date'], lag_corr_tsla['rolling_corr'], linewidth=0.4)
# plt.plot(max_corr_index_tsla, max_corr_row_tsla['rolling_corr'], marker='o', color='red')
plt.ylabel('Lagged Correlation Coefficient')
plt.title('Lagged Correlation of Sentiment and Changes in User Holdings - TSLA')
plt.show()

Average correlation coefficient

In [ ]:
mean_correlations = pd.DataFrame(lagged_correlation_results.groupby('ticker')['rolling_corr'].mean())

plt.hist(lagged_correlation_results['rolling_corr'], bins=100, color='green', linewidth=0.1)
plt.title('Distribution of Correlation Coefficients Across Stocks')
plt.show()

Aggregate Data

In [ ]:
# create aggregated data frame
correlation_data_aggr = pd.DataFrame()

# compute aggregated filtered scores and pct_change
correlation_data_aggr['aggr_pct_change'] = shifted_data.groupby('date')['pct_change'].mean()
correlation_data_aggr['aggr_scores'] = shifted_data.groupby('date')['shifted_scores'].mean()

# compute coefficients
# cross_corr_aggr = np.correlate(correlation_data_aggr['aggr_pct_change'], correlation_data_aggr['aggr_scores'], mode='same')

mean_change_aggr = np.mean(correlation_data_aggr['aggr_pct_change'])
norm_change_aggr = correlation_data_aggr['aggr_pct_change'] - mean_change_aggr
mean_score_aggr = np.mean(correlation_data_aggr['aggr_scores'])
norm_score_aggr = correlation_data_aggr['aggr_scores'] - mean_score_aggr

rolling_corr_aggr = norm_change_aggr.rolling(window=window_size).corr(norm_score_aggr)

# append to data frame
correlation_data_aggr['corr_coeff'] = rolling_corr_aggr

# reset index
correlation_data_aggr = correlation_data_aggr.reset_index()

# define max coefficient
max_corr_row_aggr = correlation_data_aggr.loc[correlation_data_aggr['corr_coeff'].idxmax()]
max_corr_index_aggr = max_corr_row_aggr['date']

Visualise with trend line

In [ ]:
dates = correlation_data_aggr['date']
y = correlation_data_aggr['corr_coeff']

# Fit a polynomial trend line
degree = 3
coeffs = np.polyfit(np.arange(len(dates)), y, degree)
trend_line = np.polyval(coeffs, np.arange(len(dates)))

# Visualize
plt.style.use('seaborn')
plt.figure(figsize=(10, 6))
plt.plot(dates, y, linewidth=0.8, color=colors[0])
plt.plot(dates, trend_line, color=colors[1], label='Trend Line', linewidth=0.8)
plt.title('Lagged Correlation Coefficient Across Stocks')
# plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.legend()
plt.tight_layout()
plt.show()


# descriptive summary
correlation_data_aggr['corr_coeff'].describe()

plt.figure(figsize=(10,6))
plt.hist(correlation_data_aggr['corr_coeff'], bins=150, color=colors[0], edgecolor='steelblue', linewidth=0.1)
plt.title('Distribution of Correlation Coefficients Across Stocks')
plt.show()

correlation_data_aggr = correlation_data_aggr.dropna()

## Compute statistical significance of correlation coefficients

In [ ]:
from scipy.stats import ttest_1samp

def test_correlation_significance(correlations, significance_level=0.05):
    t_stat, p_value = ttest_1samp(correlations, popmean=0)
    if p_value < significance_level:
        return True  # Correlation is statistically significant
    else:
        return False  # Correlation is not statistically significant

Individual stocks

In [ ]:
grouped_data = lagged_correlation_results.groupby('ticker')

# prepare count for significant and non-significant tickers
significant = 0
not_significant = 0

for ticker, group in grouped_data:

    correlations = group['rolling_corr']
    ticker = ticker

    is_significant = test_correlation_significance(correlations)
    if is_significant:
        print("The correlation for", ticker, "is statistically significant.")
        significant += 1
    else:
        print("The correlation for", ticker, "is not statistically significant.")
        not_significant += 1

print("Number of significant correlations:", significant)
print("Number of non-significant correlations:", not_significant)

Aggregated data

In [ ]:
correlations_aggr = correlation_data_aggr['corr_coeff']

# Test significance
is_significant = test_correlation_significance(correlations_aggr)
if is_significant:
    print("The correlation is statistically significant.")
else:
    print("The correlation is not statistically significant.")

# Panel Regression

In [ ]:
# Import price and volatility of stocks

path = '/Users/meret/Desktop/Uni/Master_Thesis/Data/prices'

filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    temp_df = pd.read_csv(filename)
    temp_df['Ticker'] = filename[51:-4]
    dfs.append(temp_df)

stock_data = pd.concat(dfs, ignore_index=True)
stock_data = stock_data[['Date', 'Ticker', 'Adj Close', 'Volume']]

# add percentage change in closing prices and volume
stock_data['returns'] = stock_data.groupby('Ticker')['Adj Close'].pct_change()
stock_data['vol_pct_change'] = stock_data.groupby('Ticker')['Volume'].pct_change()
stock_data.dropna(inplace=True)

# standard deviation of percentage change
tickers = sorted(pd.unique(stock_data['Ticker']))

vola = []

for ticker in tickers:
    std = stock_data[stock_data['Ticker'] == ticker]['returns'].rolling(window=30).std()
    vola.append(std)

stock_data['price_vola'] = pd.concat(vola)

# change some formatting for later
stock_data = stock_data.rename(columns={'Date': 'date', 'Ticker': 'ticker', 'Adj Close': 'price', 'Volume': 'volume'})
stock_data['ticker'] = stock_data['ticker'].str.lower()
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Check names of tickers and change them to match the rest of the data
stock_data['ticker'].unique()
stock_data['ticker'] = stock_data['ticker'].replace({'brk-b': 'brkb', 'meta': 'fb'})

# Merge the sentiment, holdings, and stock data on date and ticker

regression_data = pd.merge(correlation_data, stock_data, on=['date', 'ticker'])
regression_data = regression_data[['date', 'ticker', 'vol_pct_change', 'returns', 'filtered_scores', 'pct_change']]

## Same-day data

In [ ]:
numeric_index = regression_data.groupby("ticker").date.transform(lambda c: np.arange(len(c)))
regression_data["numeric_index"] = numeric_index
regression_data = regression_data.set_index(["ticker","numeric_index"])

# Hybrid model including both time and entity effects
fte_model_sameday = PanelOLS.from_formula('pct_change ~ 1 + filtered_scores + returns + vol_pct_change + EntityEffects + TimeEffects', data=regression_data)
fte_results_sameday = fte_model_sameday.fit(cov_type='clustered', cluster_entity=True)

# print output of FTE model for appendix
fte_summary_sameday = fte_results_sameday.summary
print(fte_summary_sameday.as_latex())

## Lagged data

In [ ]:
# Group the data by 'ticker'
regression_data = regression_data.reset_index()
regression_data = regression_data.drop(columns={'numeric_index'})
grouped_panel_data = regression_data.groupby('ticker')

# create new data frame where volume, returns, and scores are shifted by one
    # aimed to assess how 1-day lagged variables cause change in demand

shifted_panel_data = pd.DataFrame()

# Loop through each ticker in the grouped data for one-day lag
for ticker, group_df in grouped_panel_data:

    group_df['shifted_scores'] = group_df['filtered_scores'].shift(periods=1)
    group_df['shifted_returns'] = group_df['returns'].shift(periods=1)
    group_df['shifted_volume'] = group_df['vol_pct_change'].shift(periods=1)
    shifted_panel_data = pd.concat([shifted_panel_data, group_df], ignore_index=True)

# drop NaNs generated by the shift
shifted_panel_data.isna().sum()
shifted_panel_data = shifted_panel_data.dropna()
shifted_panel_data = shifted_panel_data[['date', 'ticker', 'pct_change',
                                         'shifted_scores', 'shifted_returns', 'shifted_volume']]

grouped_panel_data = shifted_panel_data.groupby('ticker')


## Panel Regression with lagged variables - Testing different models

In [ ]:
numeric_index = shifted_panel_data.groupby("ticker").date.transform(lambda c: np.arange(len(c)))
shifted_panel_data["numeric_index"] = numeric_index
shifted_panel_data = shifted_panel_data.set_index(["ticker","numeric_index"])

# Fit the fixed effects (FE) model
fe_model = PanelOLS.from_formula('pct_change ~ 1 + shifted_scores + shifted_returns + shifted_volume + EntityEffects', data=shifted_panel_data)
fe_results = fe_model.fit(cov_type='clustered', cluster_entity=True)

# Fit the time effects (TE) model
te_model = PanelOLS.from_formula('pct_change ~ 1 + shifted_scores + shifted_returns + shifted_volume + TimeEffects', data=shifted_panel_data)
te_results = te_model.fit(cov_type='clustered', cluster_entity=True)

# Hybrid model including both time and entity effects
fte_model = PanelOLS.from_formula('pct_change ~ 1 + shifted_scores + shifted_returns + shifted_volume + EntityEffects + TimeEffects', data=shifted_panel_data)
fte_results = fte_model.fit(cov_type='clustered', cluster_entity=True)

# Use test to compare the models and assess their appropriateness
model_comparison = compare({'FE': fe_results, 'TE': te_results, 'FTE': fte_results})
print(model_comparison)

comparison_summary = model_comparison.summary
print(comparison_summary.as_latex())

# print output of FTE model for main text
fte_summary = fte_results.summary
print(fte_summary.as_latex())

# Granger Causality Test

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

significance_level = 0.05

# First, check whether data is stationary

# pct_change
X = regression_data_aggr['aggr_pct_change'].values
result = adfuller(X)
if result[1] <= significance_level:
    print("Reject H0, data does not have a unit root and is stationary.")
elif result[1] > significance_level:
    print("Fail to reject H0, data is not stationary and needs to be transformed.")

# sentiment scores
X = regression_data_aggr['aggr_scores'].values
result = adfuller(X)
if result[1] <= significance_level:
    print("Reject H0, data does not have a unit root and is stationary.")
elif result[1] > significance_level:
    print("Fail to reject H0, data is not stationary and needs to be transformed.")

## Aggregated Version

In [ ]:
# Perform the Granger causality test
granger_result = grangercausalitytests(regression_data_aggr, maxlag=1)

# Extract the p-values from the test results
p_value = granger_result[1][0]['ssr_ftest'][1]

# Print the p-values
if p_value < significance_level:
    print(f'Lag {i}: p-value = {p_value}')

# Visualise

x = regression_data_aggr.index
y1 = regression_data_aggr['aggr_pct_change']
y2 = regression_data_aggr['aggr_scores']

fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
ax1.plot(x, y1, color=colors[0], label='Holdings Change', linewidth=1.5)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(x, y2, color=colors[1], label='Sentiment Score', linewidth=1.5)

ax1.tick_params(axis='x', rotation=0, labelsize=12)
ax1.set_ylabel('Holdings Change', color=colors[0], fontsize=12)
ax1.tick_params(axis='y', rotation=0, labelcolor=colors[0])
ax1.grid(alpha=.4)

ax2.set_ylabel("Sentiment Scores", color=colors[1], fontsize=12)
ax2.tick_params(axis='y', labelcolor=colors[1])
ax2.set_title("Time Series of Aggregate Holdings Change and Sentiment Scores", fontsize=18)
fig.tight_layout()
plt.show()

## Individual stocks

In [ ]:
unique_tickers = regression_data['ticker'].unique()

for ticker in unique_tickers:

    ticker_data = regression_data[regression_data['ticker'] == ticker]

    # Extract the relevant variables
    scores = ticker_data['filtered_scores']
    pct_change = ticker_data['pct_change']

    # Perform the Granger causality test
    granger_result = grangercausalitytests(np.column_stack((scores, pct_change)), maxlag=1, verbose=False)

    # Extract the p-value from the test result
    p_value = granger_result[1][0]['ssr_ftest'][1]

    # Determine whether to accept or reject the null hypothesis based on the p-value
    if p_value < significance_level:
        result = "Reject (There is Granger causality)"
    else:
        result = "Accept (There is no Granger causality)"

    # Print the result for the current ticker
    print(f"Granger Causality Test for Ticker: {ticker}")
    print(f"Result: {result}")
    print("------------------------------------------")

# Regime Switches Using Hidden Markov Model

In [ ]:
from hmmlearn.hmm import GaussianHMM
from hmmlearn import hmm

## Individual Stocks

### Sentiment

In [ ]:
regimes_sentiment = []
grouped_sentiment_data = sentiment.groupby('ticker')

unique_tickers = sentiment['ticker'].unique()
unique_tickers = np.sort(unique_tickers)

ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):
    fig, axes = plt.subplots(5, 2, figsize=(15, 20))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        group = grouped_sentiment_data.get_group(ticker)
        X = group['filtered_scores'].values.reshape(-1, 1)

        # Build the HMM model, fit to the data and predict hidden states
        model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
        model.fit(X)
        Z = model.predict(X)
        states = pd.unique(Z)

        temp_df = pd.DataFrame()
        temp_df['ticker'] = group['ticker']
        temp_df['date'] = group['date']
        temp_df['regimes'] = Z
        regimes_sentiment.append(temp_df)

        ax = axes[idx]
        ax.set_title(f"Regimes of Filtered Sentiment Scores for {ticker}")

        for i in states:
            want = (Z == i)
            x = group["date"].iloc[want]
            y = group["filtered_scores"].iloc[want]
            ax.plot(x, y, '.', markersize=8, color=colors[i])

        ax.legend(states, fontsize=10, loc='upper left', frameon=True)
        ax.grid(True)
        ax.set_ylabel("Reshaped Filtered Sentiment Scores", fontsize=10)
        ax.xaxis.set_tick_params(labelsize=10)

    for i in range(len(ticker_group), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"regimes_sentiment_{group_idx+1}.png")
    plt.show()

regimes_sentiment = pd.concat(regimes_sentiment, ignore_index=True)

### Holdings Change

In [ ]:
regimes_holdings = []
grouped_holdings_data = holdings_change.groupby('ticker')

unique_tickers = holdings_change['ticker'].unique()
unique_tickers = np.sort(unique_tickers)

ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):
    fig, axes = plt.subplots(5, 2, figsize=(15, 20))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        group = grouped_holdings_data.get_group(ticker)
        X = group['pct_change'].values.reshape(-1, 1)

        # Build the HMM model, fit to the data and predict hidden states
        model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
        model.fit(X)
        Z = model.predict(X)
        states = pd.unique(Z)

        temp_df = pd.DataFrame()
        temp_df['ticker'] = group['ticker']
        temp_df['date'] = group['date']
        temp_df['regimes'] = Z

        regimes_holdings.append(temp_df)

        ax = axes[idx]
        ax.set_title(f"Regimes of Changes in Holdings for {ticker}")

        for i in states:
            want = (Z == i)
            x = group["date"].iloc[want]
            y = group["pct_change"].iloc[want]
            ax.plot(x, y, '.', markersize=8, color=colors[i])

        ax.legend(states, fontsize=10, loc='upper left', frameon=True)
        ax.grid(True)
        ax.set_ylabel("Reshaped Changes in Holdings", fontsize=10)
        ax.xaxis.set_tick_params(labelsize=10)

    for i in range(len(ticker_group), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"regimes_holdings_{group_idx+1}.png")
    plt.show()

regimes_holdings = pd.concat(regimes_holdings, ignore_index=True)

### Lagged Correlation Coefficients

In [ ]:
regimes_correlation = []
grouped_correlation_data = lagged_correlation_results.groupby('ticker')

unique_tickers = lagged_correlation_results['ticker'].unique()
unique_tickers = np.sort(unique_tickers)

ticker_groups = np.array_split(unique_tickers, 2)

for group_idx, ticker_group in enumerate(ticker_groups):
    fig, axes = plt.subplots(5, 2, figsize=(15, 20))
    axes = axes.flatten()

    for idx, ticker in enumerate(ticker_group):
        group = grouped_correlation_data.get_group(ticker)
        X = group['rolling_corr'].values.reshape(-1, 1)

        # Build the HMM model, fit to the data and predict hidden states
        model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
        model.fit(X)
        Z = model.predict(X)
        states = pd.unique(Z)

        temp_df = pd.DataFrame()
        temp_df['ticker'] = group['ticker']
        temp_df['date'] = group['date']
        temp_df['regimes'] = Z

        regimes_correlation.append(temp_df)

        ax = axes[idx]
        ax.set_title(f"Regimes of Lagged Correlation Coefficients for {ticker}")

        for i in states:
            want = (Z == i)
            x = group["date"].iloc[want]
            y = group["rolling_corr"].iloc[want]
            ax.plot(x, y, '.', markersize=8, color=colors[i])

        ax.legend(states, fontsize=10, loc='upper left', frameon=True)
        ax.grid(True)
        ax.set_ylabel("Reshaped Correlation Coefficients", fontsize=10)
        ax.xaxis.set_tick_params(labelsize=10)

    for i in range(len(ticker_group), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"regimes_correlation_{group_idx+1}.png")
    plt.show()

regimes_correlation = pd.concat(regimes_correlation, ignore_index=True)


## Regimes for Aggregated Data

In [ ]:
colors = sns.color_palette('Set2', n_colors=3)
dates = sentiment['date']
aggr_regimes = pd.DataFrame(dates)

### Sentiment

In [ ]:
daily_avg_sentiment = sentiment.groupby('date')['filtered_scores'].mean().reset_index()

X = daily_avg_sentiment['filtered_scores'].values.reshape(-1, 1)
model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
HMMfit_sentiment = model.fit(X)
proba_sentiment = HMMfit_sentiment.predict_proba(X)
Z = model.predict(X)
states = pd.unique(Z)

aggr_regime_sentiment = pd.Series(Z, name='sentiment')
aggr_regime_sentiment = pd.concat([aggr_regime_sentiment, daily_avg_sentiment['date']], axis=1)
aggr_regimes = aggr_regimes.merge(aggr_regime_sentiment, on='date')

daily_avg_sentiment['regimes'] = aggr_regime_sentiment['sentiment']

# Visualise

ax = axes[idx]

for i in states:
    want = (Z == i)
    x = daily_avg_sentiment["date"].iloc[want]
    y = daily_avg_sentiment["filtered_scores"].iloc[want]
    plt.plot(x, y, '.', markersize=8, color=colors[i])

plt.legend(states, fontsize=10, loc='upper left', frameon=True)
plt.ylabel("Reshaped Filtered Sentiment Scores", fontsize=10)
plt.xticks(fontsize=10)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.title("Regimes of Average Daily Filtered Sentiment Scores")
plt.savefig('regimes_avg_sentiment.png')
plt.show()

### Holdings Changes

In [ ]:
daily_avg_holdings = holdings_change.groupby('date')['pct_change'].mean().reset_index()

X = daily_avg_holdings['pct_change'].values.reshape(-1, 1)
model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
HMMfit_holdings = model.fit(X)
proba_holdings = HMMfit_holdings.predict_proba(X)
Z = model.predict(X)
states = pd.unique(Z)

aggr_regime_holdings = pd.Series(Z, name='holdings')
aggr_regime_holdings = pd.concat([aggr_regime_holdings, daily_avg_holdings['date']], axis=1)
aggr_regimes = aggr_regimes.merge(aggr_regime_holdings, on='date')

daily_avg_holdings['regimes'] = aggr_regime_holdings['holdings']

# Visualise
ax = axes[idx]

for i in states:
    want = (Z == i)
    x = daily_avg_holdings["date"].iloc[want]
    y = daily_avg_holdings["pct_change"].iloc[want]
    plt.plot(x, y, '.', markersize=8, color=colors[i])

plt.legend(states, fontsize=10, loc='upper left', frameon=True)
plt.ylabel("Reshaped Holdings Change", fontsize=10)
plt.xticks(fontsize=10)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.title("Regimes of Average Daily Changes in Holdings")
plt.savefig('regimes_avg_holdings.png')
plt.show()

### Lagged Correlation Coefficients

In [ ]:
daily_avg_correlation = correlation_data_aggr[['date', 'corr_coeff']]

X = daily_avg_correlation['corr_coeff'].values.reshape(-1, 1)
model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=42)
HMMfit_correlation = model.fit(X)
proba_correlation = HMMfit_correlation.predict_proba(X)
Z = model.predict(X)
states = pd.unique(Z)

aggr_regime_correlation = pd.Series(Z, name='correlation')
aggr_regime_correlation = pd.concat([aggr_regime_correlation, correlation_data_aggr['date']], axis=1)
aggr_regimes = aggr_regimes.merge(aggr_regime_correlation, on='date')

daily_avg_correlation['regimes'] = aggr_regime_correlation['correlation']

# Visualise
ax = axes[idx]

for i in states:
    want = (Z == i)
    x = daily_avg_correlation["date"].iloc[want]
    y = daily_avg_correlation["corr_coeff"].iloc[want]
    plt.plot(x, y, '.', markersize=8, color=colors[i])

plt.legend(states, fontsize=10, loc='upper left', frameon=True)
plt.ylabel("Reshaped Correlation Coefficient", fontsize=10)
plt.xticks(fontsize=10)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.title("Regimes of Average Daily Correlation Coefficients")
plt.savefig('regimes_avg_correlation.png')
plt.show()

## Combine Information on Regimes

In [ ]:
daily_avg_sentiment = daily_avg_sentiment.rename(columns={'regimes': 'regimes_sent'})
daily_avg_holdings = daily_avg_holdings.rename(columns={'regimes': 'regimes_hold'})
daily_avg_correlation = daily_avg_correlation.rename(columns={'regimes': 'regimes_corr'})

total_regime_data = daily_avg_sentiment.merge(daily_avg_holdings, on='date')
total_regime_data = total_regime_data.merge(daily_avg_correlation, on='date')

# Box plots
plt.figure(figsize=(10, 6))
sns.boxplot(x='regimes_sent', y='filtered_scores', data=total_regime_data)
plt.xlabel('Regime')
plt.ylabel('Sentiment Scores')
plt.title('Box Plot of Sentiment Scores by Regime')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='regimes_hold', y='pct_change', data=total_regime_data)
plt.xlabel('Regime')
plt.ylabel('Changes in User Holdings')
plt.title('Box Plot of Changes in User Holdings by Regime')
plt.show()

## Look into different regimes a bit more

In [ ]:
comparison = pd.pivot_table(total_regime_data, values=['filtered_scores', 'pct_change'],
                            index=['regimes_sent', 'regimes_hold'], aggfunc=np.mean)

total_regime_data[total_regime_data['regimes_sent'] == 0].mean()
total_regime_data[total_regime_data['regimes_sent'] == 1].mean()

total_regime_data[total_regime_data['regimes_hold'] == 0].mean()
total_regime_data[total_regime_data['regimes_hold'] == 1].mean()